In [1]:
# things we need for Natural language processing
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need form Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


TFlearn: Tt is a modular and transparent deep learning library built on top of Tensorflow. It was designed to provide a higher-level API to TensorFlow in order to facilitate and speed-up experimentations, while remaining fully transparent and compatible with it. 
Natural Language Toolkit: NLTK is a open source Python library and Stemmers remove morphological affixes from words, leaving only the word stem. which is very importent when we are building a bot, because people may intract with in the way they way they normaly speak. 
Natural Language Toolkit: NLTK is a open source Python library and Stemmers remove morphological affixes from words, leaving only the word stem. which is very importent when we are building a bot, because people may intract with in the way they way they normaly speak.

In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

Pickle: It is used for serializing and de-serializing a Python object structure. Any object in python can be pickled so that it can be saved on disk. What pickle does is that it “serialises” the object first before writing it to file. Pickling is a way to convert a python object (list, dict, etc.) into a character stream.

In [4]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tflearn.regression(net)
model = tflearn.DNN(net , tflearn.regression(net) ='tflearn_logs')

SyntaxError: keyword can't be an expression (<ipython-input-4-a9a8c719432b>, line 9)

 tflearn.input_data:  This layer is used for inputting (aka. feeding) data to a network. A TensorFlow placeholder will be used if it is supplied, otherwise a new placeholder will be created with the given shape.
 tflearn.regression(net):  The regression layer is used in TFLearn to apply a regression (linear or logistic) to the provided input. It requires to specify a TensorFlow gradient descent optimizer 'optimizer' that will minimize the provided loss function 'loss' (which calculate the errors). A metric can also be provided, to evaluate the model performance.

In [5]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

tokenization is the task of chopping it up into pieces, called tokens , perhaps at the same time throwing away certain characters, such as punctuation.  A token is an instance of a sequence of characters in some particular document that are grouped together as a useful semantic unit for processing.

In [6]:
p = bow("What subjects are you intrested in?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]
['CNN', 'Machine learning', 'RNN', 'goodbye', 'greeting', 'payments', 'subjects', 'thanks']


bow: The bag-of-words model is one of the feature extraction algorithms for text.
If we want to use text in Machine Learning algorithms, we’ll have to convert then to a numerical representation.
We convert text to a numerical representation called a feature vector. A feature vector can be as simple as a list of numbers.


In [7]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from E:\bot\model.tflearn


model is our bot_stemmer

In [8]:
# create a data structure to hold user context
context = {}
#One of the parameters of a perceptron .
#The output of the perceptron is 0 if the total net input of the perceptron is less than its threshold T, otherwise it is 1.
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model

    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [9]:
classify('Which the most suatable LM algorithem?')

[('Machine learning', 0.9464242)]

In [1]:
response('what is your name?')

NameError: name 'response' is not defined

In [11]:
response('dont is a CNN?')

explanation of cnn


In [12]:
response('CNN')

explanation of cnn


In [13]:
response('Goodbye, see you later')

Bye! Come back again soon.


In [14]:
context

{'123': 'CNN'}

In [22]:
response('What subjects are you intrested in?')

Our expertice are ML, Python and cloud computing


In [16]:
# show context
context

{'123': 'CNN'}

In [17]:

response('today')

Hello, thanks for visiting


In [18]:
classify('today')

[('greeting', 0.29712152)]

In [26]:
# clear context
response("What are diffrent typrs of RNN?", show_details=True)

tag: RNN
RNN and its explanation


In [27]:
response('what is a CNN?')
classify('what is a CNN?')

explanation of cnn


[('CNN', 0.9500333)]

In [28]:

response("thanks, your great")

Any time!
